# QAOA NN Hybrid

In this Notebook we describe the QAOA+NN architecture implementation in Python. The packages needed are the following.

In [ ]:
# Pennylane packages (for the quantum circuit)
import pennylane as qml

# TensorFlow packages (for optimizing the quantum circuit)
import tensorflow as tf

# Torch packages (for the neural network)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Other packages
import networkx as nx
import tqdm.notebook as tqdm

### Graph definition 
For the graph definition we use the package `networkx`, but any other possible option given as a closed form is valid. However, some changes should be done on the rest of the code depending on the particular election.

In [ ]:
# Just as an example, we generate a d-regular graph

def dregular_graph(n,d,mu,sigma):
    """
        n --> number of qubits
        d --> number of edges connected to one node (n x d must be an even number)
    """
    G= nx.generators.random_graphs.random_regular_graph(d,n)
    for e in list(G.edges):
        G[e[0]][e[1]]["weight"] = round(random.gauss(mu,sigma),2)
    return G

G = dregular_graph(5,4,0,2)

### Neural Network definition

Here we define our neural network together with its corresponding cost function.

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(len(G.nodes), 2*len(G.nodes), False)
        #self.bn1 = nn.BatchNorm1d(len(G.nodes))
        self.fc2 = nn.Linear(2*len(G.nodes), len(G.nodes), False)
        #self.bn1 = nn.BatchNorm1d(32)
        #self.fc3 = nn.Linear(2*len(G.nodes), len(G.nodes), False)
        nn.init.eye_(self.fc1.weight)
        nn.init.eye_(self.fc2.weight)
        #nn.init.eye_(self.fc3.weight)
        
    def forward(self, x):
        x = torch.tanh(self.fc1(x))
        x = torch.tanh(self.fc2(x))
        #x = F.tanh(self.fc3(x))
        return x

In [ ]:
def MaxCut_NN(G,x):
    adj = adjacency_matrix(G)
    return 0.5*torch.matmul(x,torch.mv(adj,x))

### QAOA definition

In this part of the code, we will define the QAOA gates, the QAOA circuit and the cost funtions employed for the minimization. Several notes about the quantum circuit:

- It's a quantum simulator, so the circuit will deliver a tensor with the exact probability distributions of each possible bitstring. These exact probabilities will be used ONLY for the circuit optimization, meaning that the Neural Network needs another circuit (defined below) for obtaining a given number of samples. This minimzation for the circuit is only efficient for the circuit minimization, as we can't reach a very big number of qubits with a classical computer. For experimental implementations, the minimzation should be performed in a similar way as the NN does.
- In this implementation, we compute the circuit gradients via backpropagation instead of using parameter shift. From a computational point of view, this seems to be faster.

In [ ]:
# Cost gate
def U_C(gamma):
    for e in list(G.edges):
        wire1 = int(e[0])
        wire2 = int(e[1])
        qml.CNOT(wires=[wire1, wire2])
        qml.RZ(G[wire1][wire2]["weight"]*gamma, wires=wire2)
        qml.CNOT(wires=[wire1, wire2])

# Mixer gate
def U_M(gamma):
    for n in list(G.nodes):
        qml.RX(gamma, wires = n)

In [ ]:
# Define the depth the circuit depth
p = 1

# Definition of the circuit together with the device
dev = qml.device('default.qubit.tf', wires = len(G.nodes))
@qml.qnode(dev, interface = "tf", diff_method = "backprop")
def circuit(gamma, beta, **kwargs):
    for i in range(len(G.nodes)):
        qml.Hadamard(wires = i)
    for j in range(p):
        U_C(gamma[j])
        U_M(beta[j])
    return qml.probs(wires = list(range(len(G.nodes))))

In [ ]:
# Cost funtion and other functions needed for the energy computation

def string_to_tens(x):
    tens = torch.zeros(len(x))
    i = 0
    for el in x:
        tens[i] = float(el)
        i+=1
    return tens

def adjacency_matrix(G):
    adj = torch.zeros((len(G.nodes),len(G.nodes)))
    for edge in G.edges:
        i = edge[0]
        j = edge[1]
        adj[i,j] = G[i][j]["weight"]
        adj[j,i] = G[j][i]["weight"]
    return adj

def MaxCut_NN_QAOA(G,x, net):
    x = 1-2*x
    z = torch.sign(net(x))
    adj = adjacency_matrix(G)
    return 0.5*torch.matmul(z,torch.mv(adj,z))

def cost_function(gamma, beta, net):
    counts = {}
    result = circuit(gamma, beta)
    # In the following line, change 2 --> your number of qubits
    for i in range(len(result[0])):
        counts[f"{i:05b}"] = result[0][i]
    E = np.array([])
    for bitstring in counts.keys():
        x = string_to_tens(bitstring)
        E = np.append(E,1*float(MaxCut_NN_QAOA(G, x, net)))
        #E += -energy*counts[bitstring]
    return sum(E*result[0])

### Statistical noisy circuit
This circuit is noisy from the statistical point of view, meaning that it doesn't deliver the exact probability distribution as the previous one, but delivers a given number of samples (defined with `shots`). This circuit will be used for the neural network optimization.

In [ ]:
dev = qml.device('default.qubit.tf', wires = len(G.nodes), analytic = False, shots = 500)
@qml.qnode(dev, interface = "tf")
def circuit_stat(gamma, beta, **kwargs):
    for i in range(len(G.nodes)):
        qml.Hadamard(wires = i)
    for j in range(p):
        U_C(gamma[j])
        U_M(beta[j])
    return [qml.sample(qml.PauliZ(i)) for i in range(len(G.nodes))]

### Optimization

Here we introduce some functions to perform the optimization and an example of the optimization employed to obtain our results.

In [ ]:
def QAOA_opt(gamma, beta, opt_QAOA, net):
    """
        opt_QAOA --> optimizer used for the QAOA part.
                     Must be a tensorflow optimizer
        net --> Defines the neural network we employ
    """
    with tf.GradientTape() as tape:
        cost = cost_function(gamma, beta, net)
    gradients = tape.gradient(cost, [gamma, beta])
    opt_QAOA.apply_gradients(zip(gradients, [gamma, beta]))
    return gamma, beta, cost

def NN_opt(net, gamma, beta, opt_NN):
    """
        net --> Defines the neural network we employ
        opt_NN --> Optimizer used for the NN part.
                   Must be a torch optimizer
    """
    result = circuit_stat(gamma,beta).numpy()
    nodes, shots = np.shape(result)
    E = []
    opt_NN.zero_grad()
    for i in range(shots):
        el = [np.float(result[j][i]) for j in range(nodes)]
        input = torch.tensor(el)
        x = net(input)
        E.append(MaxCut_NN(G,x))
    cost = sum(E)/shots
    cost.backward()
    opt_NN.step()

In [ ]:
# Example of optimization

net = Net() # Initialize NN
opt_QAOA = tf.keras.optimizers.SGD(learning_rate=0.5, momentum = 0.7) # Define the QAOA optimizer
opt_NN = optim.SGD(net.parameters(), lr = 0.05) # Define the NN optimizer

# Initialize gamma and beta
gamma = tf.Variable([4.35], dtype=tf.float64)
beta = tf.Variable([0.75], dtype=tf.float64)

# Steps
steps_QAOA1 = 55 # First QAOA optimization
steps_NN = 55 # NN optimization
steps_QAOA2 = 55 # Last QAOA optimization

print("First QAOA optimization")
for k in tqdm(range(steps_QAOA1)):
    gamma, beta, cost = QAOA_opt(gamma, beta, opt_QAOA, net)
    gamma_init.append(float(gamma))
    beta_init.append(float(beta))
    en_QAOA.append(cost)

print("Neural Network optimization")
for k in tqdm(range(steps_NN)):
    NN_opt(net, gamma, beta, opt_NN)
    en_NN.append(float(checker(gamma,beta,net)))

print("Second QAOA optimization")
for k in tqdm(range(steps_QAOA2)):
    gamma, beta, cost = QAOA_opt(gamma, beta, opt_QAOA2, net)
    gamma_after.append(float(gamma))
    beta_after.append(float(beta))
    en_QAOA.append(cost)